See http://connectivity.brain-map.org/projection/experiment/168095756

In [2]:
import json
from allensdk.api.api import Api
from allensdk.api.queries.rma_api import RmaApi
from allensdk.api.queries.mouse_connectivity_api import MouseConnectivityApi
import pandas as pd
import itertools as it

rma = RmaApi()
mca = MouseConnectivityApi()

def read_data(parsed_json):
    return parsed_json['msg']

def pretty(result):
    print(json.dumps(result, indent=2))
    
def tabular_dataframe(parsed_tabular_json):
    df = pd.DataFrame.from_records(parsed_tabular_json,
                                   columns=parsed_tabular_json[0].keys())
    return df

In [3]:
section_data_set_id = 168095756

In [4]:
# rma version from detail page
results = mca.get_manual_injection_summary(section_data_set_id)

print('Manual Injection Summary')
print('Experiment: %d' % (results[0]['id']))
injections = results[0]['specimen']['injections']
print('Primary Structure: %s' % (injections[0]['structure']['name']))

print('Coords. (AP, ML, DV, L):')
for injection in injections:
    print ('\t%s(%f, %f, %f, %f)' % (injection['registration_point'],
                                     injection['coordinates_ap'],
                                     injection['coordinates_ml'],
                                     injection['coordinates_dv'],
                                     injection['angle']))
print('Transgenic Line: %s' % (results[0]['specimen']['donor']['transgenic_mouse']['transgenic_lines'][0]['name']))
print(results[0]['specimen']['donor']['transgenic_mouse']['transgenic_lines'][0]['description'])



Manual Injection Summary
Experiment: 168095756
Primary Structure: Agranular insular area, posterior part
Coords. (AP, ML, DV, L):
	Bregma(-0.340000, 3.750000, 3.250000, 0.000000)
	Bregma(-0.340000, 4.000000, 3.000000, 0.000000)
Transgenic Line: Syt6-Cre_KI148
Sparse, scattered expression in brain areas including the medulla, pons, and midbrain. Enriched in specific areas within thalamus, layer 6a cortex, and olfactory areas.


In [5]:
projection_results = mca.get_structure_projection_signal_statistics(section_data_set_id)
projection_df = tabular_dataframe(projection_results)
#pretty(projection_results)

AttributeError: 'MouseConnectivityApi' object has no attribute 'get_structure_projection_signal_statistics'

In [7]:
from allensdk.api.queries.ontologies_api import OntologiesApi
oa = OntologiesApi()

atlases = tabular_dataframe(oa.get_atlases_table(brief=True))
mouse_p56_coronal_id = atlases[atlases.name == 'Mouse, P56, Coronal']['structure_graph'].iloc[0]['id'] # 1

# calculate a list of only the ancestors that are in the limited structure set
structure_set_df = tabular_dataframe(oa.get_structures(mouse_p56_coronal_id))

oa.unpack_structure_set_ancestors(structure_set_df)

structure_set_df[structure_set_df.acronym == 'PIR']

,st_level,graph_id,name,weight,acronym,parent_structure_id,graph_order,sphinx_id,hemisphere_id,safe_name,...,failed,depth,neuro_name_structure_id_path,neuro_name_structure_id,structure_name_facet,failed_facet,id,ontology_id,atlas_id,structure_set_ancestor
377,NaN,1,Piriform area,8690,PIR,698,377,378,3,Piriform area,...,False,6,None,None,3867654445,734881840,961,1,261,"[997, 8, 567, 688, 695, 698, 961]"


In [8]:
cutoff = 0.0050
data_field = 'projection_volume'

# projection joined to structures
table = pd.concat((
    pd.merge(projection_df[projection_df.hemisphere_id == h]
                          [projection_df.projection_volume > cutoff].loc[:,['hemisphere_id',
                                                                            'structure_id',
                                                                            data_field]],
             structure_set_df[['id','acronym','graph_order']],
             how='inner',
             left_on='structure_id', right_on='id')
    for h in [1, 2])).sort('graph_order').\
        reset_index('structure_id', 'hemisphere_id')
    
table[['acronym', 'hemisphere_id', 'projection_volume']]

table

NameError: global name 'projection_df' is not defined

In [13]:
display_table = table.loc[:,['acronym', 'hemisphere_id', 'projection_volume', 'graph_order']].pivot(index='graph_order',
                                                                                                    columns='hemisphere_id').iloc[:,1:]

display_table.columns = ['structure','L', 'R']
display_table

,structure,L,R
graph_order,,,
0,root,0.020088,0.062792
1,grey,0.015301,0.058538
2,CH,NaN,0.046989
3,CTX,NaN,0.045993
4,CTXpl,NaN,0.043597
5,Isocortex,NaN,0.028033
260,AI,NaN,0.015947
267,AIp,NaN,0.009313
328,PERI,NaN,0.005093
